In [64]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import mlflow
import xgboost as xgb
from hyperopt import tpe, hp, STATUS_OK, fmin, Trials
from hyperopt.pyll import scope

In [51]:
mlflow.set_experiment('road_risk')

2025/10/14 02:55:37 INFO mlflow.tracking.fluent: Experiment with name 'road_risk' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///Users/aravindrajeshmenon/Documents/DataScienceProjects/Projects/road_accident_risk/road_risk/notebooks/mlruns/549507082084492537', creation_time=1760406937748, experiment_id='549507082084492537', last_update_time=1760406937748, lifecycle_stage='active', name='road_risk', tags={}>

In [52]:
mlflow.get_tracking_uri()

'file:///Users/aravindrajeshmenon/Documents/DataScienceProjects/Projects/road_accident_risk/road_risk/notebooks/mlruns'

In [42]:
df = pd.read_csv('../data/train.csv')

In [43]:
df = df.drop(['id'], axis = 1)

In [44]:
df.sample()

,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
314106,highway,1,0.69,25,dim,foggy,True,False,evening,True,False,2,0.35


In [45]:
x = df.drop(['accident_risk'], axis = 1)
y = df['accident_risk'].values

In [46]:
x_train, x_val, y_train, y_val= train_test_split(x,y,train_size = 0.8, random_state = 42)


In [47]:
num_cols = df.select_dtypes(['int', 'float']).columns.to_list()
cat_cols = df.select_dtypes(['object', 'bool']).columns.to_list()

num_cols.remove('accident_risk')
print(num_cols)

['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']


In [48]:
dv = DictVectorizer(sparse = False)

train_dict = x_train[cat_cols + num_cols].to_dict(orient = 'records')
x_train_encoded = dv.fit_transform(train_dict)

val_dict = x_val[cat_cols + num_cols].to_dict(orient = 'records')
x_val_encoded = dv.transform(val_dict)

In [56]:
with mlflow.start_run():
    mlflow.set_tag("model", "linear_regression")
    lr = LinearRegression()
    lr.fit(x_train_encoded, y_train)
    y_pred = lr.predict(x_val_encoded)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)
    mlflow.log_metric("rmse", rmse)

0.0735308625898314


In [61]:
with mlflow.start_run():
    mlflow.set_tag("model", "Lasso")
    lasso = Lasso(alpha = 1)
    lasso.fit(x_train_encoded, y_train)
    y_pred = lasso.predict(x_val_encoded)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)
    mlflow.log_metric("rmse", rmse)

0.16271187689427052


In [62]:
with mlflow.start_run():
    mlflow.set_tag("model", "Ridge")
    ridge = Ridge(alpha = 0.001)
    ridge.fit(x_train_encoded, y_train)
    y_pred = ridge.predict(x_val_encoded)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)
    mlflow.log_metric("rmse", rmse)
    

0.07353086258195074


In [66]:
train = xgb.DMatrix(x_train_encoded, label = y_train)
valid = xgb.DMatrix(x_val_encoded, y_val)

In [67]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params = params, 
            dtrain = train, 
            num_boost_round = 200, 
            evals=[(train, "train"), (valid, "validation")],
            early_stopping_rounds = 50,
            verbose_eval = False
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        mlflow.xgboost.log_model(booster, name = "xgb_mlflow")
        return {'loss': rmse, 'status': STATUS_OK}
        

In [70]:
search_space = {
    'max_depth' : scope.int(hp.quniform("max_depth", 10,30,1)),
    'learning_rate' : hp.loguniform("learning_rate", -3, 0),
    'min_child_weight' : hp.choice("min_samples_split", [2,5,7,10]),
    'subsample' : hp.uniform("subsample", 0.7,1),
    'colsample_bytree' : hp.uniform("colsample_bytree", 0.7,1),
    'gamma' : hp.uniform("gamma", 0, 0.5),
    'reg_alpha' : hp.loguniform("reg_alpha", -5, 0),
    "reg_lambda" : hp.loguniform("reg_lambda", -5,0),
    'objective' : 'reg:squarederror'

}

In [71]:
trials = Trials()
best_result = fmin(
    fn = objective, 
    space = search_space, 
    algo = tpe.suggest, 
    max_evals = 50
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/churnenv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [03:35:10] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/10/14 03:35:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  2%|▏         | 1/50 [00:06<05:01,  6.15s/trial, best loss: 0.05662084290306092]

/opt/anaconda3/envs/churnenv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [03:35:15] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/10/14 03:35:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



  4%|▍         | 2/50 [00:10<03:59,  5.00s/trial, best loss: 0.05645571729163345]

...

 96%|█████████▌| 48/50 [02:58<00:06,  3.02s/trial, best loss: 0.056235952632478235]

/opt/anaconda3/envs/churnenv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [03:38:08] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/10/14 03:38:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



 98%|█████████▊| 49/50 [03:01<00:02,  2.93s/trial, best loss: 0.056235952632478235]

/opt/anaconda3/envs/churnenv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [03:38:10] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)

2025/10/14 03:38:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



100%|██████████| 50/50 [03:04<00:00,  3.68s/trial, best loss: 0.056235952632478235]
